In [1]:
import pandas as pd
import re
import openpyxl
from IPython.display import display


In [4]:
from dotenv import load_dotenv
from os import getenv
from openai import OpenAI

# Load environment variables from .env file
load_dotenv("../../.env")

class chatGPT():
    def __init__(self):
        # Get API key from environment variable
        my_key = getenv("OPENAI_API_KEY")
        if my_key is None:
            raise ValueError("API key not found in .env file")
        
        # Initialize OpenAI client with API key
        self.client = OpenAI(api_key=my_key)

    def ask(self, q):
        stream = self.client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": q}],
            stream=True,
        )
        response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                response += chunk.choices[0].delta.content

        self.response = response
        return response

# Example usage
chatbot = chatGPT()
response = chatbot.ask("Hello, how are you?")
print(response)


As an artificial intelligence, I don't have feelings, but I'm here and ready to assist you. How can I help you today?


In [29]:

class ExcelHandler:
    def __init__(self):
        self.workbook = self.getWorkbook()

    def getWorkbook(self, filename='../output/tasks.xlsx'):
        try:
            workbook = openpyxl.load_workbook(filename)
        except FileNotFoundError:
            workbook = openpyxl.Workbook()
        return workbook

    def createWorksheet(self, sheet_name):
        if sheet_name in self.workbook.sheetnames:
            worksheet = self.workbook[sheet_name]
            self.workbook.remove(worksheet) 
        self.workbook.create_sheet(title=sheet_name)
        bold_font = openpyxl.styles.Font(bold=True)
        worksheet = self.workbook[sheet_name]
        worksheet.cell(row=1, column=1, value="Subtask").font = bold_font
        worksheet.cell(row=1, column=2, value="Prompt").font = bold_font
        worksheet.cell(row=1, column=3, value="Response").font = bold_font
        
        for column in worksheet.columns:
            worksheet.column_dimensions[column[0].column_letter].width = 40
        self.workbook.save('../output/tasks.xlsx')

    def getWorksheet(self, sheet_name):
        worksheet = self.workbook[sheet_name]
        return worksheet

    def writePromptsToExcel(self, sheet_name, prompts, responses, start_row=3):
        worksheet = self.getWorksheet(sheet_name)
        for row, (prompt, response) in enumerate(zip(prompts, responses), start=start_row):
            worksheet.cell(row=row, column=2, value=prompt).alignment = openpyxl.styles.Alignment(wrap_text=True)
            worksheet.cell(row=row, column=3, value=response).alignment = openpyxl.styles.Alignment(wrap_text=True)
        self.workbook.save('../output/tasks.xlsx')
    
    def writeSubtaskToExcel(self, sheet_name, task, start_row=2):
        worksheet = self.getWorksheet(sheet_name)
        worksheet.cell(row=start_row, column=1, value=task).alignment = openpyxl.styles.Alignment(wrap_text=True)
        self.workbook.save('../output/tasks.xlsx')

    def displayWorksheet(self, sheet_name):
        worksheet = self.getWorksheet(sheet_name)
        data = worksheet.values
        columns = next(data)
        df = pd.DataFrame(data, columns=columns)
        display(df)

In [30]:
class Prompting:
    def __init__(self):
        excel_file_path = '../input/prompts.xlsx'
        self.prompts = pd.read_excel(excel_file_path)
        self.chat = chatGPT()
        self.excel_handler = ExcelHandler()
        
        self.occupation = "Advertising and Promotions Manager" #"Lawyer"
        self.gov_tasks = self.getGovTask(self.occupation)
        self.chatGPTPrompting(self.occupation, self.gov_tasks)
        
    def getGovTask(self, occupation):
        df = pd.read_csv("../input/Task Statements.csv")
        occ_tasks = df.loc[df.Title == occupation + "s"]
        gov_tasks = list(occ_tasks.Task)
        return gov_tasks
        
    def splitResponse(self, response):
        return re.sub(r'\d+\.\s', '', response).split("\n")

    def getPrompt(self, i, *vars):
        prompt = self.prompts.Prompt[i - 1]
        prompt = re.sub(r'\{.*?\}', '{}', prompt)
        prompt = prompt.format(*vars)
        return prompt

    def display(self):
        sheet_name = self.occupation[:31]
        self.excel_handler.displayWorksheet(sheet_name)

        
    def chatGPTPrompting(self, occupation, tasks):
        sheet_name = occupation[:31]
        prompts = []
        responses = []
        
        self.excel_handler.createWorksheet(sheet_name)
        for task_number, task in list(enumerate(tasks, start=1))[:1]:
            self.excel_handler.writeSubtaskToExcel(sheet_name, task)
            
            prompt1 = self.getPrompt(1, occupation, task) 
            response1 = self.chat.ask(prompt1)
            prompts.append(prompt1)
            responses.append(response1)

            prompt2 = self.getPrompt(2, response1.replace("\n", "")) 
            response2 = self.chat.ask(prompt2)
            prompts.append(prompt2)
            responses.append(response2)

            subtasks = self.splitResponse(response2)
            for subtask in subtasks[:1]:
                prompt3 = self.getPrompt(3, occupation, subtask) 
                response3 = self.chat.ask(prompt3)
                prompts.append(prompt3)
                responses.append(response3)

                prompt4 = self.getPrompt(4, occupation, response3.lower()) 
                response4 = self.chat.ask(prompt4)
                prompts.append(prompt4)
                responses.append(response4)

                prompt5 = self.getPrompt
                
        self.excel_handler.writePromptsToExcel(sheet_name, task)
            

x = Prompting()


In [31]:
x.display()

Advertising and Promotions Mana


,Subtask,Prompt,Response
0,Plan and prepare advertising and promotional m...,None,None
